<div class="usecase-title">{Bins for events Argyle Squrae}</div>

<div class="usecase-authors"><b>Authored by: </b> {Alison Collins}</div>

<div class="usecase-duration"><b>Duration:</b> {60} mins</div>

<div class="usecase-level-skill">
    <div class="usecase-level"><b>Level: </b>{Intermediate}</div>
    <div class="usecase-skill"><b>Pre-requisite Skills: </b>{Python}</div>
</div>

<div class="usecase-section-header">Scenario</div>

{If you are planning an event at Argyle Square you will need to know if you need to hire additional bins. This use case seeks to determine th ebin capacity during events and make recommendations on the need for more bins based on event attendee numbers.}

<div class="usecase-section-header">What this use case will teach you</div>

At the end of this use case you will:
- {list the skills demonstrated in your use case}

<div class="usecase-section-header">{Heading for introduction or background relating to problem}</div>

{Write your introduction here. Keep it concise. We're not after "War and Peace" but enough background information to inform the reader on the rationale for solving this problem or background non-technical information that helps explain the approach. You may also wish to give information on the datasets, particularly how to source those not being imported from the client's open data portal.}



In [2]:
# Dependencies
import warnings
#warnings.filterwarnings("ignore")

# Import required modules
import requests
import numpy as np
import pandas as pd
import io
#pd.set_option('display.max_columns', None)

Import datasets using API

In [3]:
from io import StringIO

# Function to collect datasets using API
def datasetcollect(dataset_id):
    base_url = 'https://data.melbourne.vic.gov.au/api/explore/v2.1/catalog/datasets/'
    #apikey = " "
    dataset_id = dataset_id
    format = 'csv'

    url = f'{base_url}{dataset_id}/exports/{format}'
    params = {
        'select': '*',
        'limit': -1,  # all records
        'lang': 'en',
        'timezone': 'UTC',
        #'api_key': apikey
    }

    # GET request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        # StringIO to read the CSV data
        url_content = response.content.decode('utf-8')
        dataset = pd.read_csv(StringIO(url_content), delimiter=';')
        return dataset
    else:
        print(f'Request failed with status code {response.status_code}')

In [16]:
# Import stage activity dataset
dataset_id = 'meshed-sensor-type-3'
stage_activity_all = datasetcollect(dataset_id)
print(len(stage_activity_all))
stage_activity_all.head(3)

346537


,dev_id,sensor_name,time,temperature,humidity,light,motion,visit,vdd,lat_long
0,ers-55ea,NaN,2022-12-12T00:42:30+00:00,18.2,45,2299,1,1,3635,NaN
1,ers-55ea,NaN,2022-12-09T16:56:36+00:00,10.2,74,7,0,0,3635,NaN
2,ers-55eb,NaN,2022-12-09T16:50:16+00:00,10.3,74,2,0,0,3640,NaN


In [17]:
# Import bin sensor dataset
dataset_id = 'netvox-r718x-bin-sensor'
bin_sensor_all = datasetcollect(dataset_id)
print(len(bin_sensor_all))
bin_sensor_all.head(3)

555515


,dev_id,time,temperature,distance,filllevel,battery,lat_long,sensor_name,fill_level
0,r718x-6f25,2023-02-26T16:02:37+00:00,16.9,223.0,71.0,3.6,"-37.8022434, 144.9657104",r718x-bin sensor 18,69.0
1,r718x-6771,2023-02-26T08:16:37+00:00,19.1,284.0,63.0,3.6,"-37.8032085, 144.9652622",r718x-bin sensor 4,61.0
2,r718x-6f34,2023-02-26T08:21:16+00:00,20.7,380.0,51.0,3.6,"-37.802165, 144.9661423",r718x-bin sensor 20,47.0


In [18]:
# Import blix mobile phone counter dataset
dataset_id = 'blix-visits'
blix_phones_all = datasetcollect(dataset_id)
print(len(blix_phones_all))
blix_phones_all.head(3)

108311


,datetime,keys1,total,dwell,sensor_name,sensor_type,lat_long,avg_dwell
0,2022-11-14T13:00:00+00:00,7780,5,2580,Pedestrian Sensor-Argyle Sq,Mobile phone counting,"-37.8025805, 144.9656012",8.0
1,2022-11-14T17:00:00+00:00,7780,3,0,Pedestrian Sensor-Argyle Sq,Mobile phone counting,"-37.8025805, 144.9656012",0.0
2,2022-11-15T05:00:00+00:00,8472,23,20631,Pedestrian Sensor-Shrine of Remembrance - Nor...,Mobile phone counting,"-37.8303795, 144.9735707",14.0


Preprocessing of datasets

In [25]:
# Delete unwanted columns from datasets

# Drop columns from stage_activity dataframe
stage_activity = stage_activity_all[['dev_id','time','motion','visit']]
# Drop columns from stage_activity dataframe
bin_sensor = bin_sensor_all[['dev_id','time','filllevel','fill_level']]
# Drop columns from stage_activity dataframe
blix_phones = blix_phones_all[['datetime','total','dwell','avg_dwell']]

In [27]:
# Filter unwanted vlaues from bin dataset 

# Keep only rows with bin sensors in the stage area
filtered_bin_sensor = bin_sensor[bin_sensor["dev_id"].isin(["r718x-6778", "r718x-6775","r718x-6f25","r718x-677e","r718x-6f31"])]
filtered_bin_sensor.head(10)

,dev_id,time,filllevel,fill_level
0,r718x-6f25,2023-02-26T16:02:37+00:00,71.0,69.0
4,r718x-6f31,2023-02-26T08:28:41+00:00,74.0,72.0
10,r718x-6778,2023-02-26T08:46:54+00:00,73.0,71.0
11,r718x-6f25,2023-02-26T08:47:44+00:00,74.0,72.0
14,r718x-6775,2023-02-26T08:55:51+00:00,52.0,48.0
18,r718x-6f31,2023-02-26T08:58:41+00:00,73.0,71.0
22,r718x-6775,2023-02-26T09:10:51+00:00,63.0,60.0
25,r718x-6f31,2023-02-26T09:13:48+00:00,73.0,71.0
34,r718x-6f31,2023-02-26T09:43:34+00:00,74.0,72.0
36,r718x-6778,2023-02-26T09:46:54+00:00,73.0,71.0


In [42]:
# Check max and min values in bin fill levels

# Max and min of fill_level column
print(filtered_bin_sensor['fill_level'].agg(['min', 'max']))
# Max and min of filllevel column
print(filtered_bin_sensor['filllevel'].agg(['min', 'max']))


min   -8877.0
max      72.0
Name: fill_level, dtype: float64
min      0.0
max    255.0
Name: filllevel, dtype: float64


In [48]:
# Count the number of values greater than 100 and less than 0 in the bin fill coumns

print("values less than 0 in fill_level") 
print(len(filtered_bin_sensor[filtered_bin_sensor['fill_level']<0]))

print("values greater than 100 in filllevel") 
print(len(filtered_bin_sensor[filtered_bin_sensor['filllevel']>100]))


values less than 0 in fill_level
311
values greater than 100 in filllevel
61


In [ ]:
# Deal with too high and too low percentage values in bin fill value columns

In [33]:
# Count the number missing from each column in each dataframe
def count_missing_data(df):
    print(df.isnull().sum())
    return

print("number of data missing in stage activity are:")
count_missing_data(stage_activity)
print("number of data missing in bin sensor are:")
count_missing_data(filtered_bin_sensor)
print("number of data missing in blix phones are:")
count_missing_data(blix_phones)


number of data missing in stage activity are:
dev_id    0
time      0
motion    0
visit     0
dtype: int64
number of data missing in bin sensor are:
dev_id        0
time          0
filllevel     2
fill_level    2
dtype: int64
number of data missing in blix phones are:
datetime         0
total            0
dwell            0
avg_dwell    28544
dtype: int64


In [ ]:
# Deal with missing data
# 

In [ ]:
# next steps 
##### learn time series
# How to deal with data to have it all in same time intervals (10 miuntes? Hour? half hour?)
# or maybe user interactive option????
##### Work out how to deal with missing data in columns
# Use some sort of before and after combo for fill level?
# Check out dataset to see where data is missing and what it looks like? To see what will work
# Check if empty fill level = 0 as well
# and




In [21]:
!jupyter nbconvert --to html usecase_TEMPLATE_COPY.ipynb

This application is used to convert notebook files (*.ipynb)

[NbConvertApp] WARNING | pattern 'usecase_TEMPLATE_COPY.ipynb' matched no files



        to various other formats.


Options
The options below are convenience aliases to configurable class-options,
as listed in the "Equivalent to" description-line of the aliases.
To see all configurable class-options for some <cmd>, use:
    <cmd> --help-all

--debug
    set log level to logging.DEBUG (maximize logging output)
    Equivalent to: [--Application.log_level=10]
--show-config
    Show the application's configuration (human-readable format)
    Equivalent to: [--Application.show_config=True]
--show-config-json
    Show the application's configuration (json format)
    Equivalent to: [--Application.show_config_json=True]
--generate-config
    generate default config file
    Equivalent to: [--JupyterApp.generate_config=True]
-y
    Answer yes to any questions instead of prompting.
    Equivalent to: [--JupyterApp.answer_yes=True]
--execute
    Execute the notebook prior to export.
    Equivalent to: [--ExecutePreprocessor.enabled=True]
--allow-errors
    Continue noteboo